<a href="https://colab.research.google.com/github/sebastianloaa/escuela/blob/main/Anova_con_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ANOVA de un factor

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt
import seaborn as sns

### Los datos

In [ ]:
tiempo = np.array([62, 60, 63, 59, 63, 67, 71, 64, 65, 66, 68, 66, 71, 67, 68, 68, 56, 62, 60, 61, 63, 64, 63, 59])
tratamiento = np.array(["A", "A", "A", "A", "B", "B", "B", "B", "B", "B", "C", "C", "C", "C", "C", "C", "D", "D", "D", "D", "D", "D", "D", "D"])
datos = pd.DataFrame({'tiempo': tiempo, 'tratamiento': tratamiento})

### Las hipótesis

$H_o: \mu_i = \mu_j$ donde $j>i$ para $i,j = 1, 2,3, 4$

$H_1: \mu_i \neq \mu_j$ para algún $i,j = 1, 2,3, 4$

$\alpha = 0.05$

### ANOVA y Regresión lineal simple

In [ ]:
model = ols('tiempo ~ C(tratamiento)', datos).fit()
model.summary()

El Estadístico de prueba F* = 13.57 y su valor p es 0.0000466, por lo que, hay evidencia suficiente para rechazar Ho y afirmar que existen por lo menos dos medias que difieren significativamente entere sí.

### Representación de los datos por tratamiento con Diagrama de Caja y bigote

In [ ]:
# Boxplot
datos.boxplot(column='tiempo', by='tratamiento', grid=False, patch_artist=True)
plt.show()


Del gráfico se puede observar que los diagrama de caja y bigote de tratamiento AD se traslapan, lo mismo que los tratamiento BC, ¿pero será significativo? La prueba de Tukey podría ayudar en esto.  

**Complemento:** Otro gráfico de caja y bigote (que podría sustituir al anterior), pero con los datos sobrepuestos representados por puntos:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
sns.boxplot(x="tratamiento", y="tiempo", data=datos, ax=ax)
sns.swarmplot(x="tratamiento", y="tiempo", data=datos, color='black', alpha = 0.5, ax=ax);

## Prueba de Tukey para las diferencias de medias

La prueba de Tukye ofrece un resultado de diferencia de medias con un valor p ajustado

In [ ]:
# Diferencias de medias (Tukey HSD)
tukey_result = pairwise_tukeyhsd(endog=tiempo, groups=tratamiento, alpha=0.05)
print(tukey_result)

Gráfico de los resultados:

In [ ]:
tukey_result.plot_simultaneous(comparison_name='D', ylabel='Tratamiento', xlabel='Diferencia de medias')
plt.show()

In [ ]:
# Por lo que podemos concluir que los tratamientos AD son similares, lo mismo BC, pero AC difieren significatibamente de BC, a un nivel de significación de 0.05.